In [ ]:
%matplotlib inline

# Pandas

Pandas es la solución que Python propone frente a R. Es una buena tecnología para tratar conjuntos de datos "pequeños", 
es decir, aquellos que caben en la memória RAM del ordenador.

El concepto básico que introduce pandas es el de **data frame**.

## Conceptos básicos de Pandas

### Data Frames

Como una tabla, con filas y columnas (por ejemplo, como en SQL). 
Excepto:
   - Las filas pueden ser indexadas por algo interesante (hay soporte especial para etiquetas como datos categóricos y de series temporales). Esto es especialmente útil cuando tiene datos de series temporales con puntos de datos potencialmente faltantes.
   - Las celdas pueden almacenar objetos de Python. Al igual que en un excel, las columnas son homogéneas.
   - En lugar de "NULL", el nombre para un valor inexistente es "NA". A diferencia de R, los dataframes de Python solo admiten NA en columnas de algunos tipos de datos (básicamente: números en coma flotante y 'objetos'), pero esto no es un problema en su mayor parte.
  
### Serie de datos:

Estas son las columnas de un DataFrame (más correctamente, un dataframe es un diccionario de Series). Las observaciones de la serie tienen un tipo homogéneo.

<img src="img/4-pandas/Data_Frame_Data_Series.png">

In [ ]:
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

#import re # expresiones regulares

# un data frame
df1 = pd.DataFrame({
    'number': [1, 2, 3],
    'animal': ['cat', 'dog', 'mouse']
})

df1

In [ ]:
# seleccionamos la columna animal
df1['animal']

In [ ]:
df1.animal

In [ ]:
# mostramos los tipos de datos de las columnas
df1.dtypes

In [ ]:
df1['number'] = df1['number'].astype(float)

In [ ]:
df1

In [ ]:
# Creamos un dataframe igual
df2 = pd.DataFrame([
    (1.0, 'cat'),
    (2.0, 'dog'),
    (3.0, 'mouse'),
], columns=['number', 'animal'])

np.all(df1 == df2)

## Verbos (operaciones) en Pandas
  
Pandas proporciona un análisis básico de datos "con pilas incluidas":
  - ** Cargando datos: ** `read_csv`,` read_table`, `read_sql`, y ` read_html`
  - ** Selección, filtrado y agregación ** (es decir, operaciones de tipo SQL): hay una sintaxis especial para seleccionar. Existe el método `merge`. También hay una sintaxis fácil para crear una columna nueva cuyo valor se calcula desde otra columna, con la ventaja que los cálculos pueden usar toda la potencia de Python (aunque podría ser más rápido si no lo hiciera ;) ).
  - ** Agregación de estilo de "tabla pivote": ** Si eres un experto en Excel, puedes apreciar esto.
  - ** NA handling: ** Al igual que los dataframes de R, hay un buen soporte para transformar los valores de NA con valores por defecto / trucos de promedios / etc.
  - ** Estadísticas básicas: ** `mean`, ` median`, `max`,` min`, y el siempre útil `describe`.
  - ** Conectando a análisis más avanzados: ** Esto no se incluye por defecto. Pero aún así, se entiende razonablemente bien con `sklearn`.
  - ** Visualización: ** Por ejemplo `plot` y `hist`. Tambien podemos usar matplotlib y también seaborn.
  
Examinaremos un poco sobre todos estos en el contexto de un ejemplo.

Vamos a explorar un conjunto de datos de seguro hipotecario emitido por la Autoridad Federal de Vivienda (FHA). Los datos se desglosan por tramo censal y nos dice qué tamaño tiene la FHA en cada tramo (cuántas casas, etc.).

## Carga de datos y primera toma de contacto <a id='carga'></a>

**Podemos leer un csv y decirle que nombre tiene cada columna**

In [ ]:
names =["State_Code", "County_Code", "Census_Tract_Number",
        "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL",
        "AMT_FHA", "PCT_AMT_FHA"]

df = pd.read_csv('fha_by_tract.csv', names=names)  # Cargamos un archivo csv

df.head()

**O también asignar los nombres a las columnas a posteriori**

In [ ]:

names =["StateCode", "County_Code", "Census_Tract_Number",
        "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL",
        "AMT_FHA", "PCT_AMT_FHA"]

df.columns = names


In [ ]:

df = df.rename(columns={"StateCode":"State_Code"}) # incluso lo podemos hacer una columna en particular

In [ ]:
df.head() #Probad de pasar un número por parámetro

In [ ]:
df.shape

**Crear una nueva columna como combinación de otras:** 'Census_Tract_Number', 'County_Code' y 'State_Code'


In [ ]:
df['GEOID'] = df['Census_Tract_Number']*100 + 10**6 * df['County_Code'] + 10**9 * df['State_Code']   
df.head()

Si queremos eliminar una columna:

In [ ]:
column_to_drop = 'GEOID'
df.drop(column_to_drop, axis = 1).head()

La mayoría de las operaciones producen copias (a menos que se especifique `inplace = True`). El objeto `df` todavía tiene la columna GEOID.

In [ ]:
# miramos si la columna aún esta en el dataframe
column_to_drop in df.columns

In [ ]:
# Usar inplace=True no es recomendable, es mejor asignar el nuevo dataframe a una nueva variable.

df_new = df.drop(column_to_drop, axis = 1)

print(column_to_drop in df.columns)

print(column_to_drop in df_new.columns)

Las filas también se pueden eliminar. Los índices no se reinician. El índice está asociado con la fila, no con el orden.

In [ ]:
df.drop(0, axis=0).head()

Por defecto, las filas están indexadas por su posición. Sin embargo, cualquier columna se puede convertir en un índice:

In [ ]:
df.set_index('State_Code').head()

Podemos indexar a multiples niveles:

In [ ]:
df.set_index(['State_Code', 'County_Code']).head(10)

Y podemos volver atrás:

In [ ]:
df.set_index('State_Code').reset_index().head()

In [ ]:
# Podemos describir una columna
print("Percentage of mortages in each census tract insured by FHA")
df['PCT_AMT_FHA'].describe()

In [ ]:
# O el dataframe entero
df.describe()

In [ ]:
%matplotlib inline

In [ ]:
# Dibujar el histograma de una columna.
df['PCT_AMT_FHA'].plot(kind='hist', bins=100);#Probad a poner y quitar el ;

## Indexando un dataframe

Indexar por un nombre de columna produce una serie de datos.

In [ ]:
df['State_Code'].head()

Indexar por una lista de nombres de columna da otro dataframe.

In [ ]:
df[['State_Code', 'County_Code']].head()

**Pregunta:** Que nos devolverá?

In [ ]:
type(df[['State_Code']])

In [ ]:
df[['State_Code']].head()

Un dataframe es un iterador que nos devuelve el nombre de las columnas

In [ ]:
for col in df:
    print(col)

# De forma avanzada, generamos una lista de columnas
[col for col in df]

Hasta ahora hemos visto como seleccionar columnas, pero no hemos visto como seleccionar filas de nuestro conjunto de datos

In [ ]:
df[:3]

Para indexar un elemento particular del dataframe, usaremos el atributo `.loc`.  Que toma como parámetroíndice y columna.

In [ ]:
auxf = df.set_index('State_Code')
auxf.loc[28.0,:].head(10)

In [ ]:
df.loc[3, 'State_Code']

Inusualmente para Python, ambos puntos finales se incluyen en el sector.

In [ ]:
df.loc[0:3, ['State_Code','Census_Tract_Number']]

La indexación basada en la posición se realiza con el atributo `.iloc`.

In [ ]:
df.iloc[3, 0:3]

La convención de corte habitual se utiliza para `.iloc`. Es decir, el extremo superior no se incluye en el segmento

In [ ]:
df.iloc[0:3, 0:3]

## Filtrado de información

 La notación `df [...]` es muy flexible:
   - Acepta nombres de columnas (cadenas y listas de cadenas);
   - Acepta los números de las columnas (siempre que no haya ambigüedad con los nombres de las columnas);
   - ¡Acepta series de datos binarias! 
  
Esto significa que puedes escribir:
```python

 df[ df['column_name2'] == 'MD' & ( df['column_name1']==5 | df['column_name1']==6 ) ]
```   
para los que sepan SQL seria una cosa muy similar a:
```sql
SELECT * FROM df
WHERE column_name2="MD" AND (column_name1=5 OR column_name1=6)
```           
Los operadores booleanos en un dataframe devuelven una serie de datos de bools.

In [ ]:
(df['State_Code'] == 1).head()

Estos se pueden combinar con los operadores booleanos (a nivel de bit). Tened en cuenta que, debido a la precedencia del operador, es mejor poner las comparaciones individuales entre paréntesis.

In [ ]:
((df['State_Code'] == 1) & (df['Census_Tract_Number'] == 9613)).head()

Los dataframes pueden ser indexados por series de booleanos

In [ ]:
df[df['State_Code'] == 5][['State_Code', 'County_Code']].head()

** Nota: ** selecciona filas por series de datos binarios solo si comparten el mismo índice de datos.

### **Ejercicio:**
1. 

## Uniendo datos (joining)

Lo análogo a
>             
    SELECT * 
        FROM df1
        INNER JOIN df2 
        ON df1.field_name=df2.field_name;

es

    df_joined = df1.merge(df2, on='field_name')

También  se pueden hacer joins izquierda / derecha, mezclar y combinar nombres de columna, etc. [Documentación Pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)

Vamos a realizar la unión de dos dataframes

In [ ]:
df.head()

In [ ]:
# The first row is the column names, so we don't have to specify those
df_geo = pd.read_csv('2013_Gaz_tracts_national.tsv', sep='\t')
df_geo.head()



In [ ]:
df_joined = df.merge(df_geo, on="GEOID", how='left')
df_joined.head()

## Agregando información

El análogo al SQL `GROUP BY` es

    grouped = df.groupby(['field_name1', ...])...

En SQL seria

>   SELECT mean(df.value1), std(df.value2) 
        FROM df
        GROUP BY df.field_name1, ...

Pandas es algo más flexible en cuanto a cómo agrupar los datos.

In [ ]:
usps_groups = df_joined.groupby('USPS')
usps_groups

La razón por la que Pandas no requiere que especifiques una función de agregación por adelantado es porque el método groupby por sí solo ya realiza parte del trabajo. Devuelve un tipo de datos `DataFrameGroupBy` que contiene un diccionario de claves de grupo para las listas.

In [ ]:
print(type(usps_groups.groups))
usps_groups.groups['MS'][:5] # Vemos los 5 primeros



In [ ]:
usps_groups.groups.keys()

Podemos recuperar el grupo de datos asociados con una clave:

In [ ]:
usps_groups.get_group('AK')[:5]

Esto es lo mismo que:

In [ ]:
df_joined.iloc[usps_groups.groups['AK'][:5]]

In [ ]:
usps_groups.mean().head()  #Proporciona la media de cada grupo

In [ ]:
df_by_state = df_joined.groupby('USPS').sum()

In [ ]:
df_by_state.head()

También puedes especificar una función de agregación para cada columna:

In [ ]:
usps_groups['NUM_FHA', 'NUM_ALL'].agg({'NUM_FHA': np.sum, 'NUM_ALL': np.mean}).head()

La función **groupby** es especialmente útil cuando definimos nuestras propias funciones de agregación. Aquí, definimos una función que devuelve la fila para la pista del censo ubicada más al norte. La función de aplicar intenta 'combinar resultados de una manera inteligente'. La lista de objetos Serie de cada llamada a `farthest_north` para cada código USPS se contrae en una sola tabla DataFrame.

In [ ]:
def farthest_north(state_df):
    # descending sort, then select row 0
    # the datatype will be a pandas Series
    return state_df.sort_values('INTPTLAT', ascending=False).iloc[0]

df_joined.groupby('USPS').apply(farthest_north)[:10]

## Ordenando por índices y columnas

Podemos ordenar nuestro dataframe en base al índice


In [ ]:
df_by_state.sort_index(ascending=True).head()

También podemos ordenar según el valor de una columna

In [ ]:
df_by_state.sort_values('AMT_FHA', ascending=False).head()

## Valores únicos


En Pandas podemos contar el número de valores únicos, repeticiones i testear la pertenencia:

In [ ]:
df['State_Code'].unique() #[:10]


In [ ]:
aux = df['State_Code'].value_counts()

In [ ]:
df[df['State_Code'].isin(df['State_Code'].head(3))].head() #Seleccionamos según la pertenencia

## Tratamiento de información faltante y NA

Cuando leemos un  archivo CSV o una base de datos SQL, a menudo encontramos valores "NA" (o "nulo", "Ninguno", etc.). El lector CSV tiene un campo especial para especificar cómo se denota esto, y SQL tiene la noción incorporada de NULL. Pandas proporciona algunas herramientas para trabajar con estos; generalmente son similares a (y un poco peor que) `R`.

Debemos tener en cuenta que estos métodos no lo hacen ´inplace´, es decir, crean una nueva serie y no cambian la original.

[Mas detalles](http://pandas.pydata.org/pandas-docs/stable/missing_data.html)

In [ ]:
df['GEOID'][:10]

`.isnull()` y `.notnull()` examinan si existe algún null y devuelven una lista.

In [ ]:
df['GEOID'].isnull()[:10]

`.dropna()`  elimina las filas con información nula.

In [ ]:
# Comparamos el número de observaciones con NA y el número de observaciones una vez eliminamos los NA
print(df['GEOID'].size, df['GEOID'].dropna().size)

`.fillna()` substituye los valores N/A con otro valor.  `.interpolate()` substituye los valores nulos con una interpolación (linear, or quadratic, or...). 

In [ ]:
df['fill_0'] = df['GEOID'].fillna(0)                          # Fills constant value, here 0
df['fill_forward'] = df['GEOID'].fillna(method='ffill')       # Fill forwards
df['fill_back'] = df['GEOID'].fillna(method='bfill', limit=5) # Fill backwards, at most 5
df['fill_mean'] = df['GEOID'].fillna(df['GEOID'].mean())      # Fills constant value, here the mean (imputation)
df['fill_interp'] = df['GEOID'].interpolate()                 # Fills interpolated value
df[['GEOID', 'fill_0', 'fill_forward', 'fill_back', 'fill_mean', 'fill_interp']][:10]

### Nota
Los valores N / A (generalmente) se ignoran inteligentemente al realizar otros cálculos en dataframes. Por ejemplo, cuando se usan métodos de cadena en series:

In [ ]:
text_series = df['GEOID'].replace(0, np.nan).apply(str)
print(text_series[:10])

In [ ]:
text_series[:10].str.split('.')

La aplicación de la media en los datos numéricos ignora los NA por defecto (consultar la documentación):

In [ ]:
df['GEOID'].mean()

In [ ]:
states = df_joined['USPS'].dropna()
states[states.str.contains('A')].head()


# EXTRA!!

## Indices en Pandas

Los índices de pandas nos permiten manejar los datos de forma natural. ** Como hemos comentado antes, los elementos se asocian en función de su índice, no de su orden. **

In [ ]:
s1 = pd.Series([1,2,3], index=['a', 'b', 'c'])
s2 = pd.Series([3,2,1], index=['c', 'b', 'a'])
s1 + s2

In [ ]:
s3 = pd.Series([3,2,1], index=['c', 'd', 'e'])
s1 + s3

Los valores faltantes obtienen un NaN, pero esto puede ser reemplazado por un valor de relleno de nuestra elección.

In [ ]:
s1.add(s3, fill_value=0)

## Aplicando funciones

Para la aplicación de funciones a nivel de elemento, lo más sencillo es aplicar funciones **numpy** a estos objetos:

In [ ]:
df1 = pd.DataFrame(np.arange(24).reshape(4,6))

np.sin(df1)

Esto se basa en funciones numpy que se transmiten automáticamente para trabajar en función de los elementos. Para aplicar una función Python a cada elemento, use el método `.applymap ()`.

In [ ]:
df1.applymap(lambda x: "%.2f" % x)

Sin embargo, a veces desea calcular cosas en columnas o filas. En este caso, deberá usar el método `apply`.

Por ejemplo, el siguiente código muestra el rango de valores de cada columna.

In [ ]:
df1.apply(lambda x: x.max() - x.min())

## Realizando análisis más avanzados

Casi cualquier herramienta de "análisis avanzado" en el ecosistema de Python va a tomar matrices de tipo `np.array` como entrada. Puede acceder a la matriz subyacente de una columna de un data frame como

         df ['column']. values
        
Muchos de ellos toman `nd.array` a cuyos datos subyacentes se puede acceder mediante

         df.values
        
directamente. * La mayoría * de las veces, tomarán `df ['column']` y `df` sin necesidad de mirar los valores.

Esto es particularmente importante si desea usar Pandas con la biblioteca sklearn. Consultad esta [publicación](http://www.markhneedham.com/blog/2013/11/09/python-making-scikit-learn-and-pandas-play-nice/) para ver un ejemplo.

In [ ]:
df1.apply(lambda x: x.max() - x.min(), axis=1)

## Pandas Timestamps

Pandas viene con excelentes herramientas para administrar datos temporales. El elemento central de esto es la clase Timestamp, que puede inferir marcas de tiempo de muchas entradas diferentes:

In [ ]:
print(pd.Timestamp('July 4, 2016'))
print(pd.Timestamp('Monday, July 4, 2016'))
print(pd.Timestamp('Tuesday, July 4th, 2016'))  # notice it ignored 'Tuesday'
print(pd.Timestamp('Monday, July 4th, 2016 05:00 PM'))
print(pd.Timestamp('04/07/2016T17:20:13.123456'))
print(pd.Timestamp(1467651600000000000))  # number of ns since the epoch, 1/1/1970

Tambien con zonas horarias:

In [ ]:
july4 = pd.Timestamp('Monday, July 4th, 2016 05:00 PM').tz_localize('US/Eastern')
labor_day = pd.Timestamp('9/5/2016 12:00', tz='US/Eastern')
thanksgiving = pd.Timestamp('11/24/2016 16:00')  # no timezone

Pandas puede hacer cálculos en Timestamps si están localizados en la misma zona horaria o ninguno tiene una zona horaria.

In [ ]:
print(labor_day - july4)
# con  thanksgiving - july4  # obtendriamos un error

Los desplazamientos de series de tiempo son útiles para calcular fechas relativas a otra fecha. Observad que omite durante los días de fin de semana, pero es ajeno a las vacaciones. Pandas admite [calendarios personalizados] (http://pandas.pydata.org/pandas-docs/stable/timeseries.html#holidays-holiday-calendars) si los necesitamos.

In [ ]:
from pandas.tseries.offsets import BDay, Day, BMonthEnd

print(july4 + Day(5))  # 5 calendar days later, a Saturday.
print(july4 + BDay(5))  # 5 business days later, or the following Monday.
print(july4 - BDay(1))  # 1 business day earlier, or the previous Friday.
print(july4 + BMonthEnd(1))  # last business day of the month.

Pandas puede generar rangos de fechas. Aquí generamos una lista de los días de trabajo de enero de 2016:

In [ ]:
business_days = pd.date_range('1/1/2016', '1/31/2016', freq='B')
business_days

Esto a su vez puede usarse como un índice de DataFrame:

In [ ]:
time_df = pd.DataFrame(np.random.rand(len(business_days)),
                    index=business_days,
                   columns=['random'])
time_df.head()

Las funciones de zona horaria siguen siendo usables:

In [ ]:
time_df.tz_localize('UTC').tz_convert('US/Pacific').head()

## Multi-indices, stacking, and pivot tables

Data frames pueden contener múltiples índices para filas o columnas. Por ejemplo, la agrupación por dos columnas producirá un índice de fila de dos niveles.

In [ ]:
grouped = df.groupby(['State_Code', 'County_Code'])[['NUM_ALL', 'NUM_FHA']].sum()
grouped.head()

Un índice de fila pues ser transformado a un índice de columna con el método `.unstack()`:

In [ ]:
grouped.unstack().head()

El método `.stack()` realiza el trabajo contrario:

In [ ]:
np.all(grouped.unstack().stack() == grouped)

Esto puede llevarse a cabo con la función `pivot_table()` function.

In [ ]:
pd.pivot_table(df, index='State_Code', columns='County_Code',
               values=['NUM_ALL', 'NUM_FHA'], aggfunc=np.sum).head()

You may already by familiar with pivot tables in Excel.  These work similarly, and area  good tool for changing the dependent and independent variables for aggregations of data. See http://pandas.pydata.org/pandas-docs/stable/reshaping.html for more information.

### Pandas HTML data import example

Pandas takes a "batteries included" approach and throws in a whole lot of convenience functions.  For instance it has import functions for a variety of formats.  One of the pleasant surprises is a command `read_html` that's meant to automate the process of extracting tabular data from HTML.  In particular, it works pretty well with tables on Wikipedia.  

Let's do an example: We'll try to extract the list of the world's tallest structures from
http://en.wikipedia.org/wiki/List_of_tallest_buildings_and_structures_in_the_world.

In [ ]:
dfs = pd.read_html('http://en.wikipedia.org/wiki/List_of_tallest_buildings_and_structures_in_the_world', header=0, parse_dates=False)

# There are several tables on the page.  By inspection we can figure out which one we want
tallest = dfs[3]
print(tallest.columns)
# The coordinates column needs to be fixed up.  This is a bit of string parsing:
def clean_lat_long(s):
    try:
        parts = s.split("/")
    except AttributeError:
        return (None, None)
    if len(parts) < 3:
        return (None, None)
    m = re.search(r"(\d+[.]\d+);[^\d]*(\d+[.]\d+)[^\d]", parts[2])
    if not m:
        return (None, None)
    return (m.group(1), m.group(2))

tallest['Clean_Coordinates'] = tallest['Coordinates'].apply(clean_lat_long)
tallest['Latitude'] = tallest['Clean_Coordinates'].apply(lambda x:x[0])
tallest['Longitude'] = tallest['Clean_Coordinates'].apply(lambda x:x[1])

# Et voila
tallest.head()

**Exercise**

1. Parse the table rankings of [UK universities available on Wikipedia](https://en.wikipedia.org/wiki/Rankings_of_universities_in_the_United_Kingdom):